## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



In [2]:
pip install Jupytext

Note: you may need to restart the kernel to use updated packages.


___
### Tags: #Tables

### Links: 

[[t_subscribtions_citizens_by_companies_and_cities_address_ch]]

[[uk_addresses_st_partner]]

[[uk_st_partner]]

[[uk_dir_partner]]

[[entries_installation_points_dir_partner]]
___

In [32]:
query_text = """--sql
CREATE TABLE db1.t_uk_adresses_subscribtions
(
    `report_date` Date,
    `partner_uuid_uk` String,
    `name` String,
    `address_uuid` String,
    `partner_uuid` String,
    `partner_uk_email` String,
    `company_name` String,
    `partner_lk` UInt32,
    `activated_citizen_id` UInt32,
    `citizen_id_in_flat_with_subscriptions` UInt32,
    `subscribed_citizen_id` UInt32,
    `payments_amount` UInt32,
    `city` String,
    `full_address` String,
    `flats_count` UInt16,
    `entrance_number` String,
    `building_number` String,
    `streat_name` String 
)
    ENGINE = MergeTree()
    ORDER BY report_date
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [33]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_uk_adresses_subscribtions_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 35 MINUTE TO db1.t_uk_adresses_subscribtions AS
WITH d_range AS (
SELECT
	report_date
FROM db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
),
uk_st AS(
SELECT
	`report_date`,
     uk_s_p.`partner_uuid` AS partner_uuid_uk,
    `business_partner_uuid` AS partner_uuid,
    `updated_at`,
    `partner_uk_email`,
    `name`
FROM db1.uk_st_partner_ch AS uk_s_p
LEFT JOIN db1.uk_dir_partner_ch AS uk_d_p ON uk_d_p.partner_uuid_uk = uk_s_p.partner_uuid
WHERE report_date IN d_range
)
--
SELECT
    uk_addr.`report_date` AS report_date,
    uk_addr.`partner_uuid_uk` AS partner_uuid_uk,
    `name`,
    uk_addr.`address_uuid` AS address_uuid,
    uk_st.partner_uuid AS partner_uuid,
    partner_uk_email,
    cdp.company_name AS company_name,
    cdp.partner_lk AS partner_lk,
    activated_citizen_id,
    citizen_id_in_flat_with_subscriptions,
    subscribed_citizen_id,
    payments_amount,
    t_sub.city AS city,
    t_sub.full_address AS full_address,
    flats_count,
    `entrance_number`,
    `building_number`,
    `streat_name`    
FROM db1.uk_addresses_st_partner_ch AS uk_addr
LEFT JOIN uk_st ON uk_st.report_date = uk_addr.report_date 
				AND uk_st.partner_uuid_uk = uk_addr.partner_uuid_uk 
LEFT JOIN db1.companies_dir_partner_ch AS cdp ON cdp.partner_uuid = uk_st.partner_uuid
LEFT JOIN db1.t_subscribtions_citizens_by_companies_and_cities_address_ch AS t_sub
	ON t_sub.address_uuid = uk_addr.address_uuid
	AND t_sub.report_date = uk_addr.report_date
LEFT JOIN db1.entries_installation_points_dir_partner_ch AS enr_d_p 
    ON enr_d_p.address_uuid = uk_addr.address_uuid 
WHERE uk_addr.report_date in d_range
ORDER BY report_date DESC
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [2]:
query_text = """--sql
    SELECT
        *
    FROM db1.t_uk_adresses_subscribtions
    ORDER BY report_date
    limit 10
    """

ch.query_run(query_text)


,report_date,partner_uuid_uk,name,address_uuid,partner_uuid,partner_uk_email,company_name,partner_lk,activated_citizen_id,citizen_id_in_flat_with_subscriptions,subscribed_citizen_id,payments_amount,city,full_address,flats_count,entrance_number,building_number,streat_name
0,2024-01-17,8960e959-e288-40e2-90b7-86e7af975670,Проверка,eda99ac6-1d1b-4eda-9d14-c9f1a0185a7d,7407f002-183d-42eb-b25d-a666f6430365,,ООО ВидеоСервис ПРО,144422,0,0,0,0,Ломоносов,"Россия, Санкт-Петербург, Петродворцовый район,...",15,п. 7,д. 16,улица Костылева
1,2024-01-17,8960e959-e288-40e2-90b7-86e7af975670,Проверка,efa0e0f1-dec6-4ec1-a7b0-24d9bc0927a5,7407f002-183d-42eb-b25d-a666f6430365,,ООО ВидеоСервис ПРО,144422,2,0,0,0,Ломоносов,"Россия, Санкт-Петербург, Петродворцовый район,...",15,п. 6,д. 16,улица Костылева
2,2024-01-17,8960e959-e288-40e2-90b7-86e7af975670,Проверка,15d8e5e5-2da8-4cef-a7e0-fc3fcda249ca,7407f002-183d-42eb-b25d-a666f6430365,,ООО ВидеоСервис ПРО,144422,2,0,0,0,Ломоносов,"Россия, Санкт-Петербург, Петродворцовый район,...",15,п. 5,д. 16,улица Костылева
3,2024-01-17,8960e959-e288-40e2-90b7-86e7af975670,Проверка,83c7320f-00a1-4049-a954-24dc809621a4,7407f002-183d-42eb-b25d-a666f6430365,,ООО ВидеоСервис ПРО,144422,10,0,0,0,Ломоносов,"Россия, Санкт-Петербург, Петродворцовый район,...",15,п. 4,д. 16,улица Костылева
4,2024-01-17,8960e959-e288-40e2-90b7-86e7af975670,Проверка,11d6a2dd-8ee2-4212-8c5c-ea3ccb290a97,7407f002-183d-42eb-b25d-a666f6430365,,ООО ВидеоСервис ПРО,144422,11,0,0,0,Ломоносов,"Россия, Санкт-Петербург, Петродворцовый район,...",15,п. 2,д. 16,улица Костылева
5,2024-01-17,8960e959-e288-40e2-90b7-86e7af975670,Проверка,bf3b6eb2-3a9a-47f2-8753-66a57bd61b83,7407f002-183d-42eb-b25d-a666f6430365,,ООО ВидеоСервис ПРО,144422,0,0,0,0,Ломоносов,"Россия, Санкт-Петербург, Петродворцовый район,...",15,п. 1,д. 16,улица Костылева
6,2024-01-17,8aa908a4-3e36-433e-a966-b8910cecb287,ТСЖ 9000,720e4a96-14f7-47bc-b156-0acdc988913d,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,,Спутник,120012,1,0,0,0,Горно-Алтайск,"Россия, Республика Алтай, городской округ Горн...",5,п. 1,д. 39/15,улица Григория Чорос-Гуркина
7,2024-01-17,8aa908a4-3e36-433e-a966-b8910cecb287,ТСЖ 9000,b8f0a623-0978-4e9c-8240-8fe38be3d958,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,,Спутник,120012,0,0,0,0,Казань,"Россия, Республика Татарстан, городской округ ...",4,п. 3,д. 3А,ул. Алексея Козина
8,2024-01-17,117cb6e2-6f8a-432b-92f7-2093f9be9b4d,Домофон УК,40c66e6f-8253-4c40-bdec-be5301697c3d,963e8bcc-c6c3-46ee-958f-412c226ac672,,ООО Bид Плюс ПРО,120696,39,0,0,0,Озерск,"Россия, Челябинская область, Озерский городско...",44,п. 2,д. 10,улица Семёнова
9,2024-01-17,117cb6e2-6f8a-432b-92f7-2093f9be9b4d,Домофон УК,d04f8ac7-3c45-4c8e-8836-e2f4a77af034,963e8bcc-c6c3-46ee-958f-412c226ac672,,ООО Bид Плюс ПРО,120696,24,0,0,0,Озерск,"Россия, Челябинская область, Озерский городско...",36,п. 2,д. 7,проспект Карла Маркса


In [4]:
query_text = """--sql
    SELECT
        report_date,
        sum(activated_citizen_id),
        sum(citizen_id_in_flat_with_subscriptions),
        sum(subscribed_citizen_id),
        sum(payments_amount),
    FROM db1.t_uk_adresses_subscribtions
    group by report_date
    ORDER BY report_date DESC
    limit 10
    """

ch.query_run(query_text)


,report_date,sum(activated_citizen_id),sum(citizen_id_in_flat_with_subscriptions),sum(subscribed_citizen_id),sum(payments_amount)
0,2025-09-07,0,0,0,0
1,2025-09-06,0,8341,3864,14781
2,2025-09-05,0,8356,3874,13833
3,2025-09-04,0,8323,3856,14513
4,2025-09-03,0,8371,3876,13473
5,2025-09-02,0,8342,3865,14852
6,2025-09-01,0,8312,3851,14168
7,2025-08-31,50626,8313,3846,9074
8,2025-08-30,50590,8278,3834,13949
9,2025-08-29,50572,8267,3831,11876


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.t_uk_adresses_subscribtions DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [30]:
query_text = """--sql
    DROP TABLE db1.t_uk_adresses_subscribtions_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [31]:
query_text = """--sql
    DROP TABLE db1.t_uk_adresses_subscribtions
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.t_uk_adresses_subscribtions_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
897986,"""58d516b8-81ce-47ef-ab2c-c0196a…"
